In [35]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々

import numpy as np

from torchsummary import summary

In [2]:
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
print(output.size())

torch.Size([128, 30])


In [4]:
# pool of square window of size=3, stride=2
m = nn.MaxPool2d(3, stride=2)
# pool of non-square window
m = nn.MaxPool2d((3, 2), stride=(2, 1))
input = torch.randn(20, 16, 50, 32)
output = m(input)
print(output.size())

torch.Size([20, 16, 24, 31])


In [5]:
# With square kernels and equal stride
m = nn.Conv2d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# non-square kernels and unequal stride and with padding and dilation
m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.randn(20, 16, 50, 100)
output = m(input)
print(output.size())

torch.Size([20, 33, 26, 100])


In [33]:
import math

def print_shape(kernel, stride = 1, padding = 0, dilation = 1):
    offset = math.floor((2 * padding - dilation * (kernel - 1) - 1) / stride + 1)
    print("H_out = H_in / " + str(stride), '{:+}'.format(offset))


In [93]:
print_shape(4, 2, 1)

H_out = H_in / 2 +0


In [96]:
print_shape(5, 2, 2)

H_out = H_in / 2 +0


In [92]:
input = torch.rand(16, 3, 32, 32)

conv1 = nn.Conv2d(3, 6, 5) # [3, 32, 32] => [6, 28, 28]
conv2 = nn.Conv2d(6, 16, 5) # [6, 28, 28] => [16, 24, 24]
pool = nn.MaxPool2d(2, 2) # [N,C,H,W] => [N, C, H/2, W/2]
fc1 = nn.Linear(16 * 5 * 5, 120) # 
fc2 = nn.Linear(120, 84)
fc3 = nn.Linear(84, 10)
relu = F.relu

pipe = [conv1, relu, pool, conv2, relu, pool]
pipe2 = [fc1, relu, fc2, relu, fc3]

def print_pipe(pipe, input):
    print(tuple(input.shape))
    for layer in pipe:
        input = layer(input)
        print("")
        print("↓", layer)
        try:
            print("↓ weight:", tuple(layer.weight.shape))
            print("↓ bias:", tuple(layer.bias.shape))
        except AttributeError:
            pass
        print("")
        print(tuple(input.shape))
    return input
    
input = print_pipe(pipe, input)
print_pipe(pipe2, input.view(-1, 16 * 5 * 5))

(16, 3, 32, 32)

↓ Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
↓ weight: (6, 3, 5, 5)
↓ bias: (6,)

(16, 6, 28, 28)

↓ <function relu at 0x000002C7DC8F5840>

(16, 6, 28, 28)

↓ MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

(16, 6, 14, 14)

↓ Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
↓ weight: (16, 6, 5, 5)
↓ bias: (16,)

(16, 16, 10, 10)

↓ <function relu at 0x000002C7DC8F5840>

(16, 16, 10, 10)

↓ MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

(16, 16, 5, 5)
(16, 400)

↓ Linear(in_features=400, out_features=120, bias=True)
↓ weight: (120, 400)
↓ bias: (120,)

(16, 120)

↓ <function relu at 0x000002C7DC8F5840>

(16, 120)

↓ Linear(in_features=120, out_features=84, bias=True)
↓ weight: (84, 120)
↓ bias: (84,)

(16, 84)

↓ <function relu at 0x000002C7DC8F5840>

(16, 84)

↓ Linear(in_features=84, out_features=10, bias=True)
↓ weight: (10, 84)
↓ bias: (10,)

(16, 10)


tensor([[-0.0227,  0.0428, -0.0119,  0.0258,  0.0094, -0.1091, -0.0808, -0.0572,
          0.1240,  0.0965],
        [-0.0295,  0.0448, -0.0082,  0.0258,  0.0107, -0.1042, -0.0779, -0.0580,
          0.1250,  0.1018],
        [-0.0243,  0.0432, -0.0076,  0.0246,  0.0119, -0.1046, -0.0750, -0.0597,
          0.1243,  0.1062],
        [-0.0251,  0.0402, -0.0062,  0.0195,  0.0142, -0.1065, -0.0828, -0.0534,
          0.1233,  0.1038],
        [-0.0263,  0.0449, -0.0083,  0.0238,  0.0144, -0.1084, -0.0758, -0.0539,
          0.1230,  0.0970],
        [-0.0284,  0.0454, -0.0042,  0.0222,  0.0133, -0.1087, -0.0801, -0.0557,
          0.1203,  0.0992],
        [-0.0231,  0.0454, -0.0051,  0.0270,  0.0132, -0.1111, -0.0808, -0.0516,
          0.1270,  0.0927],
        [-0.0231,  0.0428, -0.0091,  0.0259,  0.0126, -0.1093, -0.0816, -0.0571,
          0.1224,  0.0985],
        [-0.0307,  0.0455, -0.0083,  0.0254,  0.0116, -0.1089, -0.0803, -0.0518,
          0.1246,  0.1022],
        [-0.0205,  

In [90]:
tuple(torch.rand(16).shape)


(16,)

In [71]:
nn.CrossEntropyLoss()(torch.randn(16, 10),
                    torch.empty(16, dtype=torch.long).random_(10))

tensor(2.5183)

In [69]:
loss = nn.CrossEntropyLoss()
input = torch.randn(32, 100, 3, 128, 128, requires_grad=True)
target = torch.empty(32, 3, 128, 128, dtype=torch.long).random_(input.shape[1])
output = loss(input, target)
output.backward()
print(input)
print(input.shape)
print(target)
print(target.shape)
print(output)

tensor([[[[[ 1.4560e+00,  1.3681e+00, -2.8873e-01,  ...,  8.6837e-01,
             6.3891e-01, -2.0169e-01],
           [-1.7223e-01,  1.7704e+00,  3.1046e-01,  ...,  5.6154e-01,
             2.7258e-01,  1.2948e+00],
           [ 1.1989e+00, -3.9140e-03,  1.2250e+00,  ..., -5.8362e-01,
            -1.7875e+00, -3.8761e-01],
           ...,
           [-6.2901e-01, -9.2756e-02, -1.8069e+00,  ..., -6.9246e-01,
            -4.8749e-01, -2.0536e-02],
           [ 6.1082e-03, -1.6717e+00,  4.7848e-01,  ..., -5.6643e-02,
            -3.7533e-01,  7.4738e-01],
           [ 2.9885e-01, -3.3261e-01, -3.8965e-01,  ...,  1.3059e-01,
            -4.8320e-01, -1.3923e+00]],

          [[-1.3461e+00,  1.0731e-01,  2.6659e-01,  ...,  1.0939e+00,
            -1.3062e-01, -2.9258e-01],
           [-1.6668e+00, -9.8555e-01, -8.8328e-02,  ..., -3.0285e-02,
             1.3723e+00,  3.4636e-01],
           [-8.0440e-01,  5.6650e-01, -5.6911e-01,  ...,  1.4593e+00,
            -2.1361e-01,  1.0786e+00],
 

             6.9365e-01, -1.7304e+00]]]]], requires_grad=True)
torch.Size([32, 100, 3, 128, 128])
tensor([[[[32, 95, 40,  ..., 59, 87, 15],
          [41, 52, 27,  ..., 10, 13, 15],
          [16, 97, 48,  ..., 92, 97, 99],
          ...,
          [59, 41, 46,  ..., 13, 47, 53],
          [86, 66, 66,  ...,  1,  0, 96],
          [76, 36, 55,  ..., 82, 68,  4]],

         [[48, 91, 73,  ..., 37, 38,  1],
          [40, 86, 44,  ..., 37,  0, 63],
          [89, 80, 58,  ..., 82, 97, 69],
          ...,
          [60, 53, 22,  ..., 32, 22, 54],
          [27, 71, 40,  ..., 28, 52, 50],
          [35, 60, 34,  ..., 28, 39, 88]],

         [[98, 63,  9,  ..., 22, 89, 88],
          [40, 68, 54,  ..., 29,  1, 99],
          [87, 39, 16,  ..., 79, 74, 44],
          ...,
          [44, 56, 32,  ...,  6, 81, 14],
          [62, 13,  6,  ..., 25, 82,  2],
          [97, 34, 73,  ...,  8,  9,  9]]],


        [[[18, 14, 65,  ..., 42, 95,  8],
          [45, 87, 17,  ..., 25, 30, 66],
         

In [70]:
loss = nn.CrossEntropyLoss()
print(loss)
nn.CrossEntropyLoss

CrossEntropyLoss()


torch.nn.modules.loss.CrossEntropyLoss

In [77]:

#model
conv1 = nn.Conv2d(3, 6, 5) # [3, 32, 32] => [6, 28, 28]
conv2 = nn.Conv2d(6, 16, 5) # [6, 28, 28] => [16, 24, 24]
pool = nn.MaxPool2d(2, 2) # [N,C,H,W] => [N, C, H/2, W/2]
fc1 = nn.Linear(16 * 5 * 5, 120) # 
fc2 = nn.Linear(120, 84)
fc3 = nn.Linear(84, 10)
relu = F.relu

pipe = [conv1, relu, pool, conv2, relu, pool]
pipe2 = [fc1, relu, fc2, relu, fc3]


#Loss関数の指定
criterion = nn.CrossEntropyLoss()

#Optimizerの指定
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

input = torch.rand(16, 3, 32, 32)
label = torch.empty(16, dtype=torch.long).random_(10)

# optimizer.zero_grad()

# outputs = model(inputs)
print("input")
input = print_pipe(pipe, input)
output = print_pipe(pipe2, input.view(-1, 16 * 5 * 5))

print("output")
print("vs label", label.shape)
loss = criterion(output, label)
loss.backward()

# optimizer.step()


input
torch.Size([16, 3, 32, 32])
↓ Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
torch.Size([16, 6, 28, 28])
↓ <function relu at 0x000002C7DC8F5840>
torch.Size([16, 6, 28, 28])
↓ MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([16, 6, 14, 14])
↓ Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
torch.Size([16, 16, 10, 10])
↓ <function relu at 0x000002C7DC8F5840>
torch.Size([16, 16, 10, 10])
↓ MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([16, 16, 5, 5])
torch.Size([16, 400])
↓ Linear(in_features=400, out_features=120, bias=True)
torch.Size([16, 120])
↓ <function relu at 0x000002C7DC8F5840>
torch.Size([16, 120])
↓ Linear(in_features=120, out_features=84, bias=True)
torch.Size([16, 84])
↓ <function relu at 0x000002C7DC8F5840>
torch.Size([16, 84])
↓ Linear(in_features=84, out_features=10, bias=True)
torch.Size([16, 10])
output
vs label torch.Size([16])


In [17]:
def debug_model(model, input):
    print(tuple(input.shape))
    for layer in model.children():
        input = layer(input)
        print("")
        print("↓", layer)
        try:
            print("↓ weight:", tuple(layer.weight.shape))
            print("↓ bias:", tuple(layer.bias.shape))
        except AttributeError:
            pass
        print("")
        print(tuple(input.shape))
    return input

#モデル定義
model = CNN()
inputs = torch.rand(16, 3, 32, 32)
debug_model(model, inputs)
# [m for m in model.children()]

model = AlexNet(3, 10)
# model.apply(lambda x: print("ap:", x))
# debug_model(model, inputs)


(16, 3, 32, 32)

↓ Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
↓ weight: (6, 3, 5, 5)
↓ bias: (6,)

(16, 6, 28, 28)

↓ Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
↓ weight: (16, 6, 5, 5)
↓ bias: (16,)

(16, 16, 24, 24)

↓ MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

(16, 16, 12, 12)


RuntimeError: size mismatch, m1: [3072 x 12], m2: [400 x 120] at C:\w\1\s\tmp_conda_3.7_100118\conda\conda-bld\pytorch_1579082551706\work\aten\src\TH/generic/THTensorMath.cpp:136

In [81]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # [3, 32, 32] => [6, 28, 28]
        self.conv2 = nn.Conv2d(6, 16, 5) # [6, 28, 28] => [16, 24, 24]
        self.pool = nn.MaxPool2d(2, 2) # [N,C,H,W] => [N, C, H/2, W/2]
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [83]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.AvgPool2d(8) #???
        )
        self.pc = nn.Sequential(
            nn.Linear(512, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 512)
#         print(x.shape)
        x = self.pc(x)
        return x

In [84]:
model = CNN()#Generator(100, 8, 3)
# inputs = torch.rand(10, 3, 32, 32)
# inputs = Variable(inputs, requires_grad=True)
# model.print_pipe(inputs)
# model(inputs).r
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
       BatchNorm2d-5           [-1, 64, 32, 32]             128
            Conv2d-6           [-1, 64, 32, 32]          36,928
              ReLU-7           [-1, 64, 32, 32]               0
         MaxPool2d-8           [-1, 64, 16, 16]               0
           Dropout-9           [-1, 64, 16, 16]               0
           Conv2d-10          [-1, 128, 16, 16]          73,856
             ReLU-11          [-1, 128, 16, 16]               0
           Conv2d-12          [-1, 128, 16, 16]         147,584
             ReLU-13          [-1, 128, 16, 16]               0
      BatchNorm2d-14          [-1, 128,